In [ ]:
%pip install google-cloud-logging

### Load app_store_id from top_chart

In [21]:
#Load the app_store_id column from the top_chart table for use in the next step of the scraping process.



import pandas as pd
from pandas_gbq import read_gbq
import logging
import google.cloud.logging
from google.cloud.logging.handlers import CloudLoggingHandler

# Initialize the Stackdriver client
client = google.cloud.logging.Client()

# Create a Cloud Logging handler for Stackdriver
cloud_handler = CloudLoggingHandler(client)

# Set up the logger
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)  # Set to INFO for standard logging, DEBUG for more detailed logs

# Add Cloud Logging and Stream handlers to the logger
stream_handler = logging.StreamHandler()
logger.addHandler(cloud_handler)  # Sends logs to Stackdriver
logger.addHandler(stream_handler)  # Sends logs to console

# Optional: Set the logging format for console output (the Cloud handler already formats for Stackdriver)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
stream_handler.setFormatter(formatter)

# Define your LIMIT and OFFSET
limit = 500
#2000
offset = 7000

# Define your BigQuery SQL query
query = f"""
    SELECT * FROM `web-scraping-2024.top_chart.top_chart`
    LIMIT {limit} OFFSET {offset}
"""

# Load the data into a Pandas DataFrame
df = read_gbq(query, project_id='web-scraping-2024')

# Generate metadata
metadata = df['app_store_id'].apply(lambda x: f"https://apps.apple.com/US/app/id{x}")

# Define messages to log and print
rows_info = f"Retrieved rows {offset + 1} to {offset + len(df)} out of total rows in the dataset."
data_shape_info = f"Current DataFrame shape: {df.shape}"

# Log and print the current position and size of the DataFrame
logger.info(rows_info)   # Logs to both Stackdriver and console
print(rows_info)         # Prints to console

logger.info(data_shape_info)  # Logs to both Stackdriver and console
print(data_shape_info)        # Prints to console

# Log the first few rows of the DataFrame at the debug level
preview_info = df.head()
logger.debug(f"DataFrame preview:\n{preview_info}")  # This log will only appear if logging level is DEBUG


Downloading: 100%|██████████|

2024-09-30 14:50:25,766 - INFO - Retrieved rows 7001 to 7500 out of total rows in the dataset.
2024-09-30 14:50:25,766 - INFO - Retrieved rows 7001 to 7500 out of total rows in the dataset.
2024-09-30 14:50:25,766 - INFO - Retrieved rows 7001 to 7500 out of total rows in the dataset.
2024-09-30 14:50:25,766 - INFO - Retrieved rows 7001 to 7500 out of total rows in the dataset.
2024-09-30 14:50:25,766 - INFO - Retrieved rows 7001 to 7500 out of total rows in the dataset.
2024-09-30 14:50:25,766 - INFO - Retrieved rows 7001 to 7500 out of total rows in the dataset.
INFO:__main__:Retrieved rows 7001 to 7500 out of total rows in the dataset.
2024-09-30 14:50:25,818 - INFO - Current DataFrame shape: (500, 12)
2024-09-30 14:50:25,818 - INFO - Current DataFrame shape: (500, 12)
2024-09-30 14:50:25,818 - INFO - Current DataFrame shape: (500, 12)
2024-09-30 14:50:25,818 - INFO - Current DataFrame shape: (500, 12)
2024-09-30 14:50:25,818 - INFO - Current DataFrame shape: (500, 12)
2024-09-30 14:


Retrieved rows 7001 to 7500 out of total rows in the dataset.
Current DataFrame shape: (500, 12)


### Scrape Data

In [22]:
# Get the data from website and save it to a dataframe as df_scraped

import requests
from bs4 import BeautifulSoup
import time
from datetime import datetime
import pandas as pd
import re
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Function to fetch data from the app store page
def fetch_app_store_data(app_store_url):
    try:
        # Send a request to the app store URL
        response = requests.get(app_store_url, allow_redirects=True)

        # If redirection occurs, get the final URL and change it to string
        scraping_url_redirect = response.url
        scraping_url_redirect = str(scraping_url_redirect) if scraping_url_redirect is not None else None  # Force to STRING

        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extract the required details
        title_with_rating = soup.find('h1', class_='product-header__title app-header__title').text.strip() if soup.find('h1', class_='product-header__title app-header__title') else None

        # Remove age rating (like "12+") from title and change it to string
        title = re.sub(r'\s*\d+\+\s*$', '', title_with_rating) if title_with_rating else None
        title = str(title) if title is not None else None  # Force to STRING

        # Scrap subtitle and change the value to string
        subtitle = soup.find('h2', class_='product-header__subtitle app-header__subtitle').text.strip() if soup.find('h2', class_='product-header__subtitle app-header__subtitle') else None
        subtitle = str(subtitle) if subtitle is not None else None  # Force to STRING

        #get avg_rating and change the value to string
        avg_rating = soup.find('figcaption', class_='we-rating-count star-rating__count').text.strip().split(' • ')[0] if soup.find('figcaption', class_='we-rating-count star-rating__count') else None
        avg_rating = str(avg_rating) if avg_rating is not None else None  # Force to STRING

        # Extract number of ratings and remove the word "Ratings" and change the value to string
        number_of_ratings = (
            soup.find('figcaption', class_='we-rating-count star-rating__count').text.strip().split(' • ')[1].replace(' Ratings', '')
            if soup.find('figcaption', class_='we-rating-count star-rating__count') else None
        )
        number_of_ratings = str(number_of_ratings) if number_of_ratings is not None else None  # Force to STRING

        # Get developer_name and change the value to string
        developer_name = soup.find('dd', class_='information-list__item__definition').text.strip() if soup.find('dd', class_='information-list__item__definition') else None
        developer_name = str(developer_name) if developer_name is not None else None  # Force to STRING

        # Get developer_url and change it to string
        developer_url = soup.find('h2', class_='product-header__identity app-header__identity').find('a')['href'] if soup.find('h2', class_='product-header__identity app-header__identity') else None
        developer_url = str(developer_url) if developer_url is not None else None  # Force to STRING

        # Extract size from the specific tag by finding the 'Size' <dt>
        size = None
        size_term = soup.find('dt', string='Size')  # Use 'string' instead of 'text'
        if size_term:
            size_definition = size_term.find_next_sibling('dd')  # Get the corresponding <dd>
            size = size_definition.text.strip() if size_definition else None
            #change the value to string
            size = str(size) if size is not None else None  # Force to STRING

        # Create a data dictionary
        data = {
            "scraping_url": str(app_store_url),
            "scraping_timestamp": datetime.now().timestamp() * 1_000_000,
            "scraping_url_redirect": str(scraping_url_redirect),
            "title": str(title) if title else '',
            "subtitle": str(subtitle) if subtitle else '',
            "avg_rating": str(avg_rating) if avg_rating else '',
            "number_of_ratings": str(number_of_ratings) if number_of_ratings else '',
            "developer_name": str(developer_name) if developer_name else '',
            "developer_url": str(developer_url) if developer_url else '',
            "size": str(size) if size else ''
        }

        logger.info(f"Successfully fetched data for {app_store_url}")
        return data

    except Exception as e:
        logger.error(f"Error fetching data for {app_store_url}: {e}")
        return None

# Example usage:
scraped_data = []
for index, row in df.iterrows():
    app_store_url = f"https://apps.apple.com/us/app/id{row['app_store_id']}"  # Assuming you build the URL like this

    # Fetch app details
    app_data = fetch_app_store_data(app_store_url)

    if app_data:
        scraped_data.append(app_data)

    # Sleep to avoid overwhelming the server
    time.sleep(1)

# Create a DataFrame from the scraped data
df_scraped = pd.DataFrame(scraped_data)

# Ensure 'scraping_timestamp' is treated as an integer for BigQuery TIMESTAMP
df_scraped['scraping_timestamp'] = df_scraped['scraping_timestamp'].astype(int)

# Ensure all other columns are treated as strings
for col in df_scraped.columns:
    if col != 'scraping_timestamp':
        df_scraped[col] = df_scraped[col].astype(str)

# Ensure columns are in the specified order
df_scraped = df_scraped[[
    "scraping_url",
    "scraping_timestamp",
    "scraping_url_redirect",
    "title",
    "subtitle",
    "avg_rating",
    "number_of_ratings",
    "developer_name",
    "developer_url",
    "size"
]]


2024-09-30 14:50:29,194 - INFO - Successfully fetched data for https://apps.apple.com/us/app/id1619316571
2024-09-30 14:50:29,194 - INFO - Successfully fetched data for https://apps.apple.com/us/app/id1619316571
2024-09-30 14:50:29,194 - INFO - Successfully fetched data for https://apps.apple.com/us/app/id1619316571
2024-09-30 14:50:29,194 - INFO - Successfully fetched data for https://apps.apple.com/us/app/id1619316571
2024-09-30 14:50:29,194 - INFO - Successfully fetched data for https://apps.apple.com/us/app/id1619316571
2024-09-30 14:50:29,194 - INFO - Successfully fetched data for https://apps.apple.com/us/app/id1619316571
INFO:__main__:Successfully fetched data for https://apps.apple.com/us/app/id1619316571
2024-09-30 14:50:30,406 - INFO - Successfully fetched data for https://apps.apple.com/us/app/id6478016068
2024-09-30 14:50:30,406 - INFO - Successfully fetched data for https://apps.apple.com/us/app/id6478016068
2024-09-30 14:50:30,406 - INFO - Successfully fetched data for ht

## Upload on GCS

In [23]:
#upload dataframe to google cloud storage. one parquet file for each app_store_id

import requests
from bs4 import BeautifulSoup
import time
from datetime import datetime
import pandas as pd
import re
from google.cloud import storage
import io
import logging

# Set up logging for StackDriver (or other destinations)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Initialize GCS client
client = storage.Client()

# Define your bucket name
bucket_name = 'app_metadata_top_chart'
bucket = client.bucket(bucket_name)

# Function to upload Parquet file to GCS inside a folder named with today's date
def upload_parquet_to_gcs(df, file_name, bucket):
    try:
        # Create an in-memory bytes buffer to save the parquet file
        buffer = io.BytesIO()

        # Write the DataFrame to Parquet format in memory
        df.to_parquet(buffer, index=False)

        # Move the buffer's position to the beginning
        buffer.seek(0)

        # Create a folder name with today's date (year_month_day)
        folder_name = datetime.now().strftime('%Y_%m_%d')

        # Create the full file path with the folder name
        file_path = f"{folder_name}/{file_name}"

        # Create a blob (file object) in the bucket with the folder and file name
        blob = bucket.blob(file_path)

        # Upload the bytes from the buffer to GCS
        blob.upload_from_file(buffer, content_type='application/octet-stream')

        message = f"Uploaded {file_path} to GCS bucket {bucket_name}"
        logger.info(message)
    except Exception as e:
        error_message = f"Failed to upload {file_name} to GCS: {e}"
        logger.error(error_message)

# Function to fetch data from the app store page
def fetch_app_store_data(app_store_url):
    try:
        # Send a request to the app store URL
        response = requests.get(app_store_url, allow_redirects=True)

        # If redirection occurs, get the final URL
        scraping_url_redirect = response.url
        scraping_url_redirect = str(scraping_url_redirect) if scraping_url_redirect is not None else None  # Force to STRING

        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extract the required details
        title_with_rating = soup.find('h1', class_='product-header__title app-header__title').text.strip() if soup.find('h1', class_='product-header__title app-header__title') else None

        # Remove age rating (like "12+") from title
        title = re.sub(r'\s*\d+\+\s*$', '', title_with_rating) if title_with_rating else None
        title = str(title) if title is not None else None  # Force to STRING

        subtitle = soup.find('h2', class_='product-header__subtitle app-header__subtitle').text.strip() if soup.find('h2', class_='product-header__subtitle app-header__subtitle') else None
        subtitle = str(subtitle) if subtitle is not None else None  # Force to STRING

        avg_rating = soup.find('figcaption', class_='we-rating-count star-rating__count').text.strip().split(' • ')[0] if soup.find('figcaption', class_='we-rating-count star-rating__count') else None
        avg_rating = str(avg_rating) if avg_rating is not None else None  # Force to STRING

        # Extract number of ratings and remove the word "Ratings"
        number_of_ratings = (
            soup.find('figcaption', class_='we-rating-count star-rating__count').text.strip().split(' • ')[1].replace(' Ratings', '')
            if soup.find('figcaption', class_='we-rating-count star-rating__count') else None
        )
        number_of_ratings = str(number_of_ratings) if number_of_ratings is not None else None  # Force to STRING


        developer_name = soup.find('dd', class_='information-list__item__definition').text.strip() if soup.find('dd', class_='information-list__item__definition') else None
        developer_name = str(developer_name) if developer_name is not None else None  # Force to STRING

        developer_url = soup.find('h2', class_='product-header__identity app-header__identity').find('a')['href'] if soup.find('h2', class_='product-header__identity app-header__identity') else None
        developer_url = str(developer_url) if developer_url is not None else None  # Force to STRING

        # Extract size from the specific tag by finding the 'Size' <dt>
        size = None
        size_term = soup.find('dt', string='Size')
        if size_term:
            size_definition = size_term.find_next_sibling('dd')
            size = size_definition.text.strip() if size_definition else None
            size = str(size) if size is not None else None  # Force to STRING

        return {
            "scraping_url": str(app_store_url) if app_store_url else None,
            "scraping_timestamp": datetime.now().timestamp() * 1_000_000,
            "scraping_url_redirect": str(scraping_url_redirect) if scraping_url_redirect else None,
            "title": str(title) if title else None,
            "subtitle": str(subtitle) if subtitle else None,
            "avg_rating": str(avg_rating) if avg_rating else None,
            "number_of_ratings": str(number_of_ratings),  # Only the numeric part
            "developer_name": str(developer_name) if developer_name else None,
            "developer_url": str(developer_url) if developer_url else None,
            "size": str(size) if size else None
        }
    except Exception as e:
        error_message = f"Error fetching data for {app_store_url}: {e}"
        logger.error(error_message)
        return None

# Assuming you have a DataFrame df with the app_store_id and other relevant columns.
scraped_data = []
for index, row in df.iterrows():
    app_store_url = f"https://apps.apple.com/us/app/id{row['app_store_id']}"  # Assuming you build the URL like this

    # Fetch app details
    app_data = fetch_app_store_data(app_store_url)

    if app_data:
        # Append to the list
        scraped_data.append(app_data)

        # Create a DataFrame for the single app data
        df_app = pd.DataFrame([app_data])

        # Define the file name using app_store_id and current date
        app_id = row['app_store_id']
        scraping_date = datetime.now().strftime('%Y_%m_%d')
        scraping_date = str(scraping_date) if scraping_date is not None else None  # Force to STRING

        file_name = f"{scraping_date}__{app_id}.parquet"

        # Upload the DataFrame to GCS inside the date-named folder
        upload_parquet_to_gcs(df_app, file_name, bucket)

    # Sleep to avoid overwhelming the server
    time.sleep(1)

# Optionally, create a DataFrame from all scraped data if needed
if scraped_data:
    df_scraped = pd.DataFrame(scraped_data)

    # Ensure columns are in the specified order
    df_scraped = df_scraped[[
        "scraping_url",
        "scraping_timestamp",
        "scraping_url_redirect",
        "title",
        "subtitle",
        "avg_rating",
        "number_of_ratings",
        "developer_name",
        "developer_url",
        "size"
    ]]


2024-09-30 15:21:56,938 - INFO - Uploaded 2024_09_30/2024_09_30__1619316571.parquet to GCS bucket app_metadata_top_chart
2024-09-30 15:21:56,938 - INFO - Uploaded 2024_09_30/2024_09_30__1619316571.parquet to GCS bucket app_metadata_top_chart
2024-09-30 15:21:56,938 - INFO - Uploaded 2024_09_30/2024_09_30__1619316571.parquet to GCS bucket app_metadata_top_chart
2024-09-30 15:21:56,938 - INFO - Uploaded 2024_09_30/2024_09_30__1619316571.parquet to GCS bucket app_metadata_top_chart
2024-09-30 15:21:56,938 - INFO - Uploaded 2024_09_30/2024_09_30__1619316571.parquet to GCS bucket app_metadata_top_chart
2024-09-30 15:21:56,938 - INFO - Uploaded 2024_09_30/2024_09_30__1619316571.parquet to GCS bucket app_metadata_top_chart
INFO:__main__:Uploaded 2024_09_30/2024_09_30__1619316571.parquet to GCS bucket app_metadata_top_chart
2024-09-30 15:21:58,589 - INFO - Uploaded 2024_09_30/2024_09_30__6478016068.parquet to GCS bucket app_metadata_top_chart
2024-09-30 15:21:58,589 - INFO - Uploaded 2024_09_3

### Load on BigQuery

In [ ]:
import os
import pandas as pd
from google.cloud import storage, bigquery
from google.cloud.bigquery import LoadJobConfig, SourceFormat
import io
import logging

# Set up Python logger
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

# Set up BigQuery client
bq_client = bigquery.Client()

# Initialize GCS client
gcs_client = storage.Client()

# Define GCS bucket and BigQuery details
bucket_name = 'app_metadata_top_chart'
project_id = 'web-scraping-2024'
dataset_id = 'app_metadata'
table_id = 'app_metada'

bucket = gcs_client.bucket(bucket_name)

# Function to load batch data from GCS Parquet files into BigQuery
def load_batch_to_bigquery(dataframes, table_id, file_index, total_files):
    try:
        # Concatenate all DataFrames in the batch
        batch_df = pd.concat(dataframes, ignore_index=True)

        # Explicitly convert the necessary columns to string type
        batch_df["scraping_url"] = batch_df["scraping_url"].astype(str)
        batch_df["scraping_timestamp"] = pd.to_datetime(batch_df["scraping_timestamp"], errors='coerce')
        batch_df["scraping_url_redirect"] = batch_df["scraping_url_redirect"].astype(str)
        batch_df["title"] = batch_df["title"].astype(str)
        batch_df["subtitle"] = batch_df["subtitle"].astype(str)
        batch_df["avg_rating"] = batch_df["avg_rating"].astype(str)
        batch_df["number_of_ratings"] = batch_df["number_of_ratings"].astype(str)
        batch_df["developer_name"] = batch_df["developer_name"].astype(str)
        batch_df["developer_url"] = batch_df["developer_url"].astype(str)
        batch_df["size"] = batch_df["size"].astype(str)

        # Write the DataFrame back to a Parquet file in memory
        buffer = io.BytesIO()
        batch_df.to_parquet(buffer, index=False)
        buffer.seek(0)

        job_config = LoadJobConfig(
            source_format=SourceFormat.PARQUET,
            write_disposition="WRITE_APPEND"  # Appends data to the table
        )

        # Load Parquet file from memory to BigQuery
        load_job = bq_client.load_table_from_file(
            buffer,
            table_id,
            job_config=job_config
        )

        # Wait for the load job to complete
        load_job.result()

        logger.info(f"Loaded {file_index}/{total_files} batch of Parquet files into BigQuery table {table_id}")
    except Exception as e:
        logger.error(f"Error loading batch data to BigQuery: {e}")

# Function to list Parquet files in a specific GCS subdirectory
def list_parquet_files_in_subdirectory(bucket, subdirectory):
    try:
        blobs = bucket.list_blobs(prefix=subdirectory)
        parquet_files = [blob.name for blob in blobs if blob.name.endswith('.parquet')]

        if parquet_files:
            logger.info(f"Found {len(parquet_files)} Parquet files in subdirectory {subdirectory}")
        else:
            logger.warning(f"No Parquet files found in subdirectory {subdirectory}")

        return parquet_files
    except Exception as e:
        logger.error(f"Error listing Parquet files in subdirectory {subdirectory}: {e}")
        return []

# Main function to load Parquet files from a specific subdirectory in GCS to BigQuery in batches
def load_parquet_files_from_subdirectory_to_bigquery(subdirectory, batch_size=10):
    # List all Parquet files in the specified GCS subdirectory
    parquet_files = list_parquet_files_in_subdirectory(bucket, subdirectory)

    # Total number of files to load
    total_files = len(parquet_files)
    batch_dataframes = []

    # Loop through each Parquet file
    for idx, file_name in enumerate(parquet_files, start=1):
        gcs_uri = f"gs://{bucket_name}/{file_name}"
        print(f"{idx}/{total_files} Loading {gcs_uri}")

        try:
            # Read the Parquet file into a DataFrame
            df = pd.read_parquet(gcs_uri)

            # Explicitly convert the necessary columns to string type before adding to batch
            df["scraping_url"] = df["scraping_url"].astype(str)
            df["scraping_timestamp"] = pd.to_datetime(df["scraping_timestamp"], errors='coerce')
            df["scraping_url_redirect"] = df["scraping_url_redirect"].astype(str)
            df["title"] = df["title"].astype(str)
            df["subtitle"] = df["subtitle"].astype(str)
            df["avg_rating"] = df["avg_rating"].astype(str)
            df["number_of_ratings"] = df["number_of_ratings"].astype(str)
            df["developer_name"] = df["developer_name"].astype(str)
            df["developer_url"] = df["developer_url"].astype(str)
            df["size"] = df["size"].astype(str)

            # Add the DataFrame to the batch
            batch_dataframes.append(df)

            # If batch size is reached or it's the last file, load the batch to BigQuery
            if len(batch_dataframes) == batch_size or idx == total_files:
                load_batch_to_bigquery(batch_dataframes, f"{project_id}.{dataset_id}.{table_id}", idx, total_files)
                batch_dataframes = []  # Reset batch

        except Exception as e:
            logger.error(f"Error reading Parquet file {gcs_uri}: {e}")

if __name__ == "__main__":
    subdirectory = "2024_09_30/"  # Specify the subdirectory to load files from
    try:
        load_parquet_files_from_subdirectory_to_bigquery(subdirectory, batch_size=10)
        logger.info("All files loaded successfully to BigQuery")
    except Exception as e:
        logger.error(f"Failed to load Parquet files to BigQuery: {e}")


2024-09-30 15:47:51,454 - INFO - Found 2258 Parquet files in subdirectory 2024_09_30/
2024-09-30 15:47:51,454 - INFO - Found 2258 Parquet files in subdirectory 2024_09_30/
2024-09-30 15:47:51,454 - INFO - Found 2258 Parquet files in subdirectory 2024_09_30/
2024-09-30 15:47:51,454 - INFO - Found 2258 Parquet files in subdirectory 2024_09_30/
2024-09-30 15:47:51,454 - INFO - Found 2258 Parquet files in subdirectory 2024_09_30/
2024-09-30 15:47:51,454 - INFO - Found 2258 Parquet files in subdirectory 2024_09_30/
INFO:__main__:Found 2258 Parquet files in subdirectory 2024_09_30/


1/2258 Loading gs://app_metadata_top_chart/2024_09_30/2024_09_30__1000136689.parquet
2/2258 Loading gs://app_metadata_top_chart/2024_09_30/2024_09_30__1000669926.parquet
3/2258 Loading gs://app_metadata_top_chart/2024_09_30/2024_09_30__1001244846.parquet
4/2258 Loading gs://app_metadata_top_chart/2024_09_30/2024_09_30__1001257338.parquet
5/2258 Loading gs://app_metadata_top_chart/2024_09_30/2024_09_30__1001473964.parquet
6/2258 Loading gs://app_metadata_top_chart/2024_09_30/2024_09_30__1001893116.parquet
7/2258 Loading gs://app_metadata_top_chart/2024_09_30/2024_09_30__1002043426.parquet
8/2258 Loading gs://app_metadata_top_chart/2024_09_30/2024_09_30__1002345611.parquet
9/2258 Loading gs://app_metadata_top_chart/2024_09_30/2024_09_30__1003820457.parquet
10/2258 Loading gs://app_metadata_top_chart/2024_09_30/2024_09_30__1004562049.parquet


2024-09-30 15:48:12,636 - INFO - Loaded 10/2258 batch of Parquet files into BigQuery table web-scraping-2024.app_metadata.app_metada
2024-09-30 15:48:12,636 - INFO - Loaded 10/2258 batch of Parquet files into BigQuery table web-scraping-2024.app_metadata.app_metada
2024-09-30 15:48:12,636 - INFO - Loaded 10/2258 batch of Parquet files into BigQuery table web-scraping-2024.app_metadata.app_metada
2024-09-30 15:48:12,636 - INFO - Loaded 10/2258 batch of Parquet files into BigQuery table web-scraping-2024.app_metadata.app_metada
2024-09-30 15:48:12,636 - INFO - Loaded 10/2258 batch of Parquet files into BigQuery table web-scraping-2024.app_metadata.app_metada
2024-09-30 15:48:12,636 - INFO - Loaded 10/2258 batch of Parquet files into BigQuery table web-scraping-2024.app_metadata.app_metada
INFO:__main__:Loaded 10/2258 batch of Parquet files into BigQuery table web-scraping-2024.app_metadata.app_metada


11/2258 Loading gs://app_metadata_top_chart/2024_09_30/2024_09_30__1004611265.parquet
12/2258 Loading gs://app_metadata_top_chart/2024_09_30/2024_09_30__1007346233.parquet
13/2258 Loading gs://app_metadata_top_chart/2024_09_30/2024_09_30__1008234539.parquet
14/2258 Loading gs://app_metadata_top_chart/2024_09_30/2024_09_30__1011285354.parquet
15/2258 Loading gs://app_metadata_top_chart/2024_09_30/2024_09_30__1012274888.parquet
16/2258 Loading gs://app_metadata_top_chart/2024_09_30/2024_09_30__1015578037.parquet
17/2258 Loading gs://app_metadata_top_chart/2024_09_30/2024_09_30__1016686482.parquet
18/2258 Loading gs://app_metadata_top_chart/2024_09_30/2024_09_30__1017148055.parquet
19/2258 Loading gs://app_metadata_top_chart/2024_09_30/2024_09_30__1017813651.parquet
20/2258 Loading gs://app_metadata_top_chart/2024_09_30/2024_09_30__1018627241.parquet


2024-09-30 15:48:34,402 - INFO - Loaded 20/2258 batch of Parquet files into BigQuery table web-scraping-2024.app_metadata.app_metada
2024-09-30 15:48:34,402 - INFO - Loaded 20/2258 batch of Parquet files into BigQuery table web-scraping-2024.app_metadata.app_metada
2024-09-30 15:48:34,402 - INFO - Loaded 20/2258 batch of Parquet files into BigQuery table web-scraping-2024.app_metadata.app_metada
2024-09-30 15:48:34,402 - INFO - Loaded 20/2258 batch of Parquet files into BigQuery table web-scraping-2024.app_metadata.app_metada
2024-09-30 15:48:34,402 - INFO - Loaded 20/2258 batch of Parquet files into BigQuery table web-scraping-2024.app_metadata.app_metada
2024-09-30 15:48:34,402 - INFO - Loaded 20/2258 batch of Parquet files into BigQuery table web-scraping-2024.app_metadata.app_metada
INFO:__main__:Loaded 20/2258 batch of Parquet files into BigQuery table web-scraping-2024.app_metadata.app_metada


...

1971/2258 Loading gs://app_metadata_top_chart/2024_09_30/2024_09_30__6503959130.parquet
1972/2258 Loading gs://app_metadata_top_chart/2024_09_30/2024_09_30__6504047941.parquet
1973/2258 Loading gs://app_metadata_top_chart/2024_09_30/2024_09_30__6504127360.parquet
1974/2258 Loading gs://app_metadata_top_chart/2024_09_30/2024_09_30__6504385346.parquet
1975/2258 Loading gs://app_metadata_top_chart/2024_09_30/2024_09_30__6504513740.parquet
1976/2258 Loading gs://app_metadata_top_chart/2024_09_30/2024_09_30__6504558978.parquet
1977/2258 Loading gs://app_metadata_top_chart/2024_09_30/2024_09_30__6504626192.parquet
1978/2258 Loading gs://app_metadata_top_chart/2024_09_30/2024_09_30__6504741161.parquet
1979/2258 Loading gs://app_metadata_top_chart/2024_09_30/2024_09_30__6504841441.parquet
1980/2258 Loading gs://app_metadata_top_chart/2024_09_30/2024_09_30__650812005.parquet


2024-09-30 16:56:27,736 - INFO - Loaded 1980/2258 batch of Parquet files into BigQuery table web-scraping-2024.app_metadata.app_metada
2024-09-30 16:56:27,736 - INFO - Loaded 1980/2258 batch of Parquet files into BigQuery table web-scraping-2024.app_metadata.app_metada
2024-09-30 16:56:27,736 - INFO - Loaded 1980/2258 batch of Parquet files into BigQuery table web-scraping-2024.app_metadata.app_metada
2024-09-30 16:56:27,736 - INFO - Loaded 1980/2258 batch of Parquet files into BigQuery table web-scraping-2024.app_metadata.app_metada
2024-09-30 16:56:27,736 - INFO - Loaded 1980/2258 batch of Parquet files into BigQuery table web-scraping-2024.app_metadata.app_metada
2024-09-30 16:56:27,736 - INFO - Loaded 1980/2258 batch of Parquet files into BigQuery table web-scraping-2024.app_metadata.app_metada
INFO:__main__:Loaded 1980/2258 batch of Parquet files into BigQuery table web-scraping-2024.app_metadata.app_metada


1981/2258 Loading gs://app_metadata_top_chart/2024_09_30/2024_09_30__6508168515.parquet
1982/2258 Loading gs://app_metadata_top_chart/2024_09_30/2024_09_30__650828103.parquet
1983/2258 Loading gs://app_metadata_top_chart/2024_09_30/2024_09_30__653124851.parquet
1984/2258 Loading gs://app_metadata_top_chart/2024_09_30/2024_09_30__654856368.parquet
1985/2258 Loading gs://app_metadata_top_chart/2024_09_30/2024_09_30__6553976921.parquet
1986/2258 Loading gs://app_metadata_top_chart/2024_09_30/2024_09_30__6553988262.parquet
1987/2258 Loading gs://app_metadata_top_chart/2024_09_30/2024_09_30__655572115.parquet
1988/2258 Loading gs://app_metadata_top_chart/2024_09_30/2024_09_30__656846983.parquet
1989/2258 Loading gs://app_metadata_top_chart/2024_09_30/2024_09_30__6572299276.parquet
1990/2258 Loading gs://app_metadata_top_chart/2024_09_30/2024_09_30__6572306062.parquet


2024-09-30 16:56:51,798 - INFO - Loaded 1990/2258 batch of Parquet files into BigQuery table web-scraping-2024.app_metadata.app_metada
2024-09-30 16:56:51,798 - INFO - Loaded 1990/2258 batch of Parquet files into BigQuery table web-scraping-2024.app_metadata.app_metada
2024-09-30 16:56:51,798 - INFO - Loaded 1990/2258 batch of Parquet files into BigQuery table web-scraping-2024.app_metadata.app_metada
2024-09-30 16:56:51,798 - INFO - Loaded 1990/2258 batch of Parquet files into BigQuery table web-scraping-2024.app_metadata.app_metada
2024-09-30 16:56:51,798 - INFO - Loaded 1990/2258 batch of Parquet files into BigQuery table web-scraping-2024.app_metadata.app_metada
2024-09-30 16:56:51,798 - INFO - Loaded 1990/2258 batch of Parquet files into BigQuery table web-scraping-2024.app_metadata.app_metada
INFO:__main__:Loaded 1990/2258 batch of Parquet files into BigQuery table web-scraping-2024.app_metadata.app_metada


1991/2258 Loading gs://app_metadata_top_chart/2024_09_30/2024_09_30__6578425605.parquet
1992/2258 Loading gs://app_metadata_top_chart/2024_09_30/2024_09_30__6581491699.parquet
1993/2258 Loading gs://app_metadata_top_chart/2024_09_30/2024_09_30__658439903.parquet
1994/2258 Loading gs://app_metadata_top_chart/2024_09_30/2024_09_30__658539297.parquet
1995/2258 Loading gs://app_metadata_top_chart/2024_09_30/2024_09_30__6587558825.parquet
1996/2258 Loading gs://app_metadata_top_chart/2024_09_30/2024_09_30__6596811159.parquet
1997/2258 Loading gs://app_metadata_top_chart/2024_09_30/2024_09_30__661910862.parquet
1998/2258 Loading gs://app_metadata_top_chart/2024_09_30/2024_09_30__663046683.parquet
